<a href="https://colab.research.google.com/github/PamePatzi/Sistema-inteligente-para-monitoreo-e-identificaci-n-temprana-de-melanoma/blob/main/MelanomaModelo1_ASAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SISTEMA INTELIGENTE PARA IDENTIFICACIÓN Y DIAGNOSTICO TEMPRANA DE MELANOMAS (Parte 1)

Para esta primera parte se presenta la creación de un modelo convolucional en el que se aplican 3 capas con las imagenes del dataset obtenido desde la página de Kaggle
#Importacion de las depependencias

Para empezar se cargan todas las librerías que se usaran en los codigos para su funcionamiento, incluyendo las que son de ayuda para la descarga del dataset.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import json
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np



Mediante la credencial de kaggle y la API especifica del dataset extraido, se procede a descargarlo y descomprimirlo en su carpeta principal, además de eliminar el .zip que luego de descomprimirlo no usaremos.

In [ ]:
# Se define las credentiales para acceder a Kaggle
!mkdir ~/.kaggle # Crear el directorio .kaggle
!touch ~/.kaggle/kaggle.json # Crear el archivo kaggle.json
api_token = {"username":"lexipatzi","key":"af4d8478480353a66efb23e233482439"} # Definir las credenciales de API de Kaggle
with open('/root/.kaggle/kaggle.json', 'w') as file: # Guardando las credenciales
    json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json # Establecer los permisos del archivo kaggle.json

In [ ]:
!kaggle datasets download -d drscarlat/melanoma # Descargar el dataset desde Kaggle
!unzip -qn '/content/melanoma.zip' -d /content/ > /dev/null # Descomprimiendo el dataset 
!rm /content/melanoma.zip # Eliminando el zip
!rm -r dermmel/ # Eliminando el DERMMEL

100% 5.24G/5.26G [00:57<00:00, 120MB/s]
100% 5.26G/5.26G [00:57<00:00, 97.5MB/s]


Siguendo con el procedimiento se pretende realizar un aumento de datos haciendo uso de data augmentation, este procedimiento es extra, ya que se cuenta con un dataset suficiente para su entrenamiento, sin embargo se considera importante y relevante considerar un dataset con más imagenes para que el modelo sea lo más funcional posible

In [ ]:
# Estas capas sólo están activas durante el entrenamiento, cuando se llama a Model.fitz
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

##Preparacion de los datos
Se realiza un pre procesado de las imagenes para que estas se encuentren en el mismo formato y puedan ser analizadas de la misma manera. Con el siguente código las imagenes pasan por un rescalamiento, además que se agrega el target_size, para que el tamaño requerido sea de (224,224). 

In [ ]:
# Preparación del conjunto de datos
#datos train
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
train_dataset = train_datagen.flow_from_directory('/content/DermMel/train_sep', target_size=(224, 224), batch_size=10,class_mode='binary')
#datos test
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
test_dataset = test_datagen.flow_from_directory('/content/DermMel/test',target_size=(224, 224),batch_size=10,class_mode='binary')
#datos validación
val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
validation_dataset = val_datagen.flow_from_directory('/content/DermMel/valid',target_size=(224, 224),batch_size=10,class_mode='binary')

Found 10682 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.


In [ ]:

# Creación del modelo CNN
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(100, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilación del modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluación del modelo
loss, accuracy = model.evaluate(validation_dataset)
print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/10
1069/1069 [==============================] - 1544s 1s/step - loss: 0.5875 - accuracy: 0.6546 - val_loss: 0.8871 - val_accuracy: 0.6698
Epoch 2/10
1069/1069 [==============================] - 1500s 1s/step - loss: 0.5203 - accuracy: 0.7243 - val_loss: 0.5223 - val_accuracy: 0.7214
Epoch 3/10
1069/1069 [==============================] - 1522s 1s/step - loss: 0.4477 - accuracy: 0.7881 - val_loss: 0.4160 - val_accuracy: 0.8141
Epoch 4/10
1069/1069 [==============================] - 1519s 1s/step - loss: 0.3987 - accuracy: 0.8191 - val_loss: 0.3895 - val_accuracy: 0.8166
Epoch 5/10
1069/1069 [==============================] - 1500s 1s/step - loss: 0.3843 - accuracy: 0.8233 - val_loss: 0.4166 - val_accuracy: 0.8116
Epoch 6/10
1069/1069 [==============================] - 1503s 1s/step - loss: 0.3593 - accuracy: 0.8400 - val_loss: 0.3640 - val_accuracy: 0.8312
Epoch 7/10
1069/1069 [==============================] - 1506s 1s/step - loss: 0.3425 - accuracy: 0.8474 - val_loss: 0.3582 -

In [ ]:
model.save('/content/model1.h5')

#Pruebas con Imagenes subidas

In [ ]:

# Carga del modelo entrenado
model = keras.models.load_model('/content/model1.h5')
#imagen de prueba
img_path = '/content/DermMel/valid/Melanoma/AUG_0_105.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
imga = image.img_to_array(img)
imga = np.expand_dims(imga, axis=0)
imga /= 255.0
#predicción
prediction = model.predict(imga)
print(prediction)
#class_idx = int(prediction[0][0])
#print("predicción",class_idx)
if prediction < 0.5: #por las caracteristicas de nuestro modelo nos dara predicciones arriba de 0.5 o abajo de este
    print("No melanoma")
else:
    print("Melanoma")

1/1 [==============================] - 0s 319ms/step
[[0.59616774]]
Melanoma


#Envio de respuesta via Telegram

In [ ]:
!pip install python-telegram-bot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.4/545.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
TOKEN = "5858768507:AAFL8yBpPmfZAeJK826YxIfLtFZXzThpKJA" # @param {type: "string"}
CHAT_ID = "5514441598" # @param {type: "string"}

In [ ]:
import requests
def bot_send_text(bot_message):
    
    bot_token = TOKEN
    bot_chatID = CHAT_ID
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response

In [ ]:
import datetime

#D = [1, 0, 1, 0] prediccion realizada
if prediction < 0.5:
    print("No melanoma")
else:
    print("Melanoma")
# Obtener la fecha actual
fecha_actual = datetime.date.today().strftime("%d-%m-%Y")

# Variables con los datos a llenar
no_orden = "" # @param {type: "string"}
nombre_paciente = "" # @param {type: "string"}
edad_paciente = "" # @param {type: "string"}
telefono = "" # @param {type: "string"}
ci = "" # @param {type: "string"}
x = ""
if prediction >0.5:
  x = "Si posee un Melanoma"
else:
  x = "No posee un Melanoma"
# Generar el texto con los datos

texto = f'''\
\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tPagina No. 01 \
\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tFecha: {fecha_actual}\
\n================================================\
\nNo Orden : {no_orden}\t\t\t\t\t\t\t\t\t\t\t\tNo. ingreso : 010\
\nPaciente : {nombre_paciente}\tEdad\t\t\t\t\t\t\t : {edad_paciente}\
\nC.I.\t\t\t\t : {ci}\t\t\t\t\t\t\tTelefono\t\t\t : {telefono}\
\nEl tiene el siguiente diagnostico: {x}
'''

# Enviar el texto al bot o imprimirlo
print(texto)

test_bot = bot_send_text(texto)

Melanoma

																											Pagina No. 01 
																											Fecha: 06-06-2023
No Orden : 												No. ingreso : 010
Paciente : 	Edad							 : 
C.I.				 : 							Telefono			 : 
El tiene el siguiente diagnostico: Si posee un Melanoma



#Prueba con cámara web


In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
def modelo(img):
  # Load and compile the model
  model = load_model('model1.h5')
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  # List of category names for the dataset CIFAR-10
  category_names = ['melanoma', 'no_melanoma']
  frame=img #load the image from the webcam
  # Resize the frame to 32x32 pixels
  frame = cv2.resize(frame, (224, 224))
  frame_array = np.array(frame) #we transform it into an array
  # Normalize the frame data
  frame_array = frame_array.astype('float32') / 255
  # Make predictions with the frame
  predictions = model.predict(frame_array)
  # Get the predicted label index
  predicted_label_index = np.argmax(predictions[0])
  # Get the predicted label value
  predicted_label_value = predictions[0][predicted_label_index]
  # Get the predicted category
  predicted_category = category_names[predicted_label_index]
  # Check if the predicted label value is greater than 0.65
  if predicted_label_value > 0.65:
      print(f"{predicted_category}: {predicted_label_value}")

In [ ]:
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

#Prueba con Cámara web en tiempo real

In [ ]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    modelo(img) #we send the image previously converted to the main function of this exercise Our result would be different predicctions made on real time.